In [ ]:
import os
# from sklearn.utils import shuffle
import time
import cv2
import numpy as np
# import tqdm
from PIL import Image
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Flatten, Dropout
from keras.layers import Input, merge
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
# from misc_layers import MinibatchDiscrimination, SubPixelUpscaling, CustomLRELU, bilinear2x
# from keras_contrib.layers import SubPixelUpscaling
import keras.backend as K
from keras.initializers import RandomNormal
# K.set_image_dim_ordering('tf')

import glob
os.environ["KERAS_BACKEND"] = "tensorflow"
# from sklearn.utils import shuffle
import scipy
import imageio
from PIL import Image
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from keras.models import load_model
import keras.backend as K
# from scipy.interpolate import spline

from os import listdir
from os.path import isfile, join

from collections import deque
np.random.seed(1337)

In [ ]:
def norm_img(img):
    img = (img / 127.5) - 1
    #image normalisation to keep values between -1 and 1 for stability
    return img

def denorm_img(img):
    #for output
    img = (img + 1) * 127.5
    return img.astype(np.uint8) 


def sample_from_dataset(batch_size, image_shape, data_dir=None, data = None):
    sample_dim = (batch_size,) + image_shape
    sample = np.empty(sample_dim, dtype=np.float32)
    
    sample_imgs_paths = [join(data_dir, f) for f in listdir(data_dir) if 'png' in f]
    sample_imgs_paths = np.random.choice(sample_imgs_paths,batch_size)
    for index,img_filename in enumerate(sample_imgs_paths):
        image = Image.open(img_filename)
        image = image.resize(image_shape[:-1])
        image = image.convert('RGB')
        image = np.asarray(image)
        image = norm_img(image)
        sample[index,...] = image
    return sample

def gen_noise(batch_size, noise_shape):
    #input noise for the generator should follow a probability distribution, like in this case, the normal distributon.
    return np.random.normal(0, 1, size=(batch_size,)+noise_shape)

In [ ]:
def get_gen_normal(noise_shape):
    noise_shape = noise_shape
    
    kernel_init = 'glorot_uniform'
    
    gen_input = Input(shape = noise_shape)
    generator = Conv2DTranspose(filters = 512, kernel_size = (4,4), strides = (1,1), padding = "valid", data_format = "channels_last", kernel_initializer = kernel_init)(gen_input)
    generator = BatchNormalization(momentum = 0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
        
    generator = Conv2DTranspose(filters = 256, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    generator = BatchNormalization(momentum = 0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    generator = Conv2DTranspose(filters = 128, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    generator = BatchNormalization(momentum = 0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    generator = Conv2DTranspose(filters = 64, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    generator = BatchNormalization(momentum = 0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    generator = Conv2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    generator = BatchNormalization(momentum = 0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    generator = Conv2DTranspose(filters = 3, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    generator = Activation('tanh')(generator)
        
    gen_opt = Adam(lr=0.00015, beta_1=0.5)
    generator_model = Model(input = gen_input, output = generator)
    generator_model.compile(loss='binary_crossentropy', optimizer=gen_opt, metrics=['accuracy'])
#     generator_model.summary()

    return generator_model



def get_disc_normal(image_shape=(64,64,3)):
    image_shape = image_shape
    
    dropout_prob = 0.2
    kernel_init = 'glorot_uniform'
    dis_input = Input(shape = image_shape)
    discriminator = Conv2D(filters = 64, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(dis_input)
    discriminator = LeakyReLU(0.2)(discriminator)
    discriminator = Conv2D(filters = 128, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(discriminator)
    discriminator = BatchNormalization(momentum = 0.5)(discriminator)
    discriminator = LeakyReLU(0.2)(discriminator)
   
    discriminator = Conv2D(filters = 256, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(discriminator)
    discriminator = BatchNormalization(momentum = 0.5)(discriminator)
    discriminator = LeakyReLU(0.2)(discriminator)
    
    discriminator = Conv2D(filters = 512, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(discriminator)
    discriminator = BatchNormalization(momentum = 0.5)(discriminator)
    discriminator = LeakyReLU(0.2)(discriminator)
    
    discriminator = Flatten()(discriminator)
    
    discriminator = Dense(1)(discriminator)
    discriminator = Activation('sigmoid')(discriminator)
    #also try the SGD optimiser, might work better for a few learning rates.
    dis_opt = Adam(lr=0.0002, beta_1=0.5)
    discriminator_model = Model(input = dis_input, output = discriminator)
    discriminator_model.compile(loss='binary_crossentropy', optimizer=dis_opt, metrics=['accuracy'])
#     discriminator_model.summary()
    return discriminator_model

In [ ]:
noise_shape = (1,1,100)
batch_size = 64
img_save_dir = "./output_source_word/"
image_shape = (64,64,3)
data_dir =  "./input_source_word/"

save_model_dir = './model_source_word/'

In [ ]:
discriminator = get_disc_normal(image_shape)
generator = get_gen_normal(noise_shape)

In [ ]:
discriminator.trainable = False

opt = Adam(lr=0.00015, beta_1=0.5) 
gen_inp = Input(shape=noise_shape)
GAN_inp = generator(gen_inp)
GAN_opt = discriminator(GAN_inp)
gan = Model(input = gen_inp, output = GAN_opt)
gan.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])
gan.summary()

In [ ]:
def get_sample_images(img_batch):
    rand_index = np.random.randint(img_batch.shape[0])
    rand_index = 0
    image = img_batch[rand_index, :,:,:]
    return denorm_img(image)

def save_img_one_batch(plt, img_batch, img_save_dir, step):
    image = get_sample_images(img_batch)
    plt.imshow(image)
    plt.savefig(img_save_dir, bbox_inches='tight', pad_inches=0)
def plot_history_loss():
    plt.figure(figsize=(20,5))
    length = 8000
    plt.ylim(0, 1.0)
    x = range(len(real_loss_array[:length]))
    plt.plot(x, real_loss_array[:length], '-r', x, fake_loss_array[:length], '-g')

In [ ]:
init_step = 0
num_steps = init_step + 10500
# batch_size = 5
# print(init_step, num_steps)

In [ ]:
avg_disc_real_loss = deque([0], maxlen=250)
avg_GAN_loss = deque([0], maxlen=250)

real_loss_array, global_loss_array = [], []

plt.figure(figsize=(4,4))
plt.axis('off')
batch_size = 5
for step in range(init_step, num_steps): 
    if ((step + 1) % 250) == 0:
        print("Begin step: ", step)
    
    # Learning D
    real_data_X = sample_from_dataset(batch_size, image_shape, data_dir = data_dir)
    noise = gen_noise(batch_size,noise_shape)
    fake_data_X = generator.predict(noise)
    
    real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0
    fake_data_Y = np.random.random_sample(batch_size)*0.01
    
#     data_X = np.concatenate([real_data_X, fake_data_X])
#     data_Y = np.concatenate((real_data_Y, fake_data_Y))
        
    discriminator.trainable = True
    generator.trainable = False
    dis_metrics_real = discriminator.train_on_batch(real_data_X, real_data_Y) 
    dis_metrics_fake = discriminator.train_on_batch(fake_data_X, fake_data_Y) 
    
    
    # Learning G
    generator.trainable = True
    discriminator.trainable = False
    GAN_X = gen_noise(batch_size,noise_shape)
    GAN_Y = real_data_Y
    gan_metrics = gan.train_on_batch(GAN_X,GAN_Y)
    
    
    # tracking 
    real_loss_array.append(dis_metrics_real[0])
    global_loss_array.append(gan_metrics[0])
    if (step+1) % 50 == 0:
        step_num = str(step).zfill(4)
        save_img_one_batch(plt, fake_data_X, img_save_dir+step_num+"_image.png", step)
        
        
    real_loss_array.append(dis_metrics_real[0])
    avg_GAN_loss.append(gan_metrics[0])
    if (((step+1) % 1000) == 0) or (step == num_steps - 1):
        print("-----------------------------------------------------------------")
        print("Average Disc_real loss: %f" % (np.mean(avg_disc_real_loss)))    
        print("Average GAN loss: %f" % (np.mean(avg_GAN_loss)))
        print("-----------------------------------------------------------------")
        discriminator.trainable = True
        generator.trainable = True
        generator.save(save_model_dir+str(step)+"_GENERATOR_weights_and_arch.hdf5")
        discriminator.save(save_model_dir+str(step)+"_DISCRIMINATOR_weights_and_arch.hdf5")


In [ ]:
plot_history_loss()

In [ ]:
print('done')